In [1]:
# import libraries
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 100)

TARGET = 'Listening_Time_minutes'
CATS = ['Podcast_Name', 'Episode_Num', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
NUMS = ['Episode_Length_minutes', 'Host_Popularity_percentage', 
        'Guest_Popularity_percentage', 'Number_of_Ads']

oof_pred_name = '2+3+4_interact_xgb'

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
train = pd.read_csv('../data/raw/train.csv', index_col='id')
test = pd.read_csv('../data/raw/test.csv', index_col='id')
original = pd.read_csv('../data/raw/podcast_dataset.csv')
print(f"Train shape: {train.shape}")
print(f"Test  shape: {test.shape}")
print(f"Orig  shape: {original.shape}")
train.head(3)

Train shape: (750000, 11)
Test  shape: (250000, 10)
Orig  shape: (52500, 11)


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
id,,,,,,,,,,,
0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.8,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.9,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531


In [4]:
original_clean = original.dropna(subset=[TARGET]).drop_duplicates()
train = pd.concat([train, original_clean], axis=0, ignore_index=True)

In [5]:
train.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,703281.000000,794868.000000,644364.000000,794867.000000,794868.000000
mean,64.413254,59.877089,52.102374,1.357405,45.443291
std,32.981496,22.888959,28.481602,1.149726,27.138465
min,0.000000,1.300000,0.000000,0.000000,0.000000
25%,35.670000,39.450000,28.130000,0.000000,23.184810
50%,63.770000,60.060000,53.360000,1.000000,43.391710
75%,94.010000,79.560000,76.500000,2.000000,64.814290
max,325.240000,119.460000,119.910000,103.910000,119.970000


Add Features

In [6]:
def feature_eng(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df['Episode_Num'] = df['Episode_Title'].str[8:]     
    df['is_weekend']   = df['Publication_Day'].isin(['Saturday', 'Sunday']).astype(int)

    return df.drop(columns=['Episode_Title'])

train = feature_eng(train)
test = feature_eng(test)

In [7]:
ELM = []
for k in range(3):
    col_name = f'ELm_r{k}'
    train[col_name] = train['Episode_Length_minutes'].round(k)
    test[col_name] = test['Episode_Length_minutes'].round(k)
    ELM.append(col_name)

In [8]:
train.columns

Index(['Podcast_Name', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Episode_Num', 'is_weekend', 'ELm_r0',
       'ELm_r1', 'ELm_r2'],
      dtype='object')

In [9]:
encoded_columns = []

selected_comb = [
    # 2-interaction
    ['Episode_Length_minutes', 'Host_Popularity_percentage'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Number_of_Ads'],
    ['Episode_Num', 'Host_Popularity_percentage'],
    ['Episode_Num', 'Guest_Popularity_percentage'],
    ['Episode_Num', 'Number_of_Ads'],    
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Host_Popularity_percentage', 'Number_of_Ads'],
    ['Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Podcast_Name'],
    ['Episode_Num', 'Podcast_Name'],  
    ['Guest_Popularity_percentage', 'Podcast_Name'],
    ['ELm_r1', 'Episode_Num'],
    ['ELm_r1', 'Host_Popularity_percentage'], 
    ['ELm_r1', 'Guest_Popularity_percentage'],
    ['ELm_r2', 'Episode_Num'],
    ['ELm_r2', 'Episode_Sentiment'],
    ['ELm_r2', 'Publication_Day'],

    
    # 3-interaction
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Num', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Num', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Num', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Num', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Sentiment', 'Publication_Time'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Genre'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Genre'],
    ['Episode_Num', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],

    ['Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],   
    ['ELm_r1', 'Number_of_Ads', 'Episode_Sentiment'],
    ['ELm_r2', 'Number_of_Ads', 'Podcast_Name'],
    
    # 4-interaction
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Num', 'Host_Popularity_percentage', 'Genre'],
    ['Episode_Length_minutes', 'Episode_Num', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Num', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Num', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Num', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Num', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Num', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Num', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Num', 'Publication_Day', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Num', 'Publication_Day', 'Genre'],    
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day', 'Genre'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],
    ['Episode_Length_minutes', 'Episode_Num', 'Publication_Time', 'Podcast_Name'],
    
    ['Episode_Num', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Publication_Day'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Publication_Time'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Genre'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Publication_Time', 'Genre'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Num', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],
    ['Episode_Num', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Time', 'Podcast_Name'],
    
]

for comb in selected_comb:
    name = '_'.join(comb)
        
    if len(comb) == 2:
        train[name] = train[comb[0]].astype(str) + '_' + train[comb[1]].astype(str)
        test[name] = test[comb[0]].astype(str) + '_' + test[comb[1]].astype(str)
        
    elif len(comb) == 3:
        train[name] = (train[comb[0]].astype(str) + '_' +
                       train[comb[1]].astype(str) + '_' +
                       train[comb[2]].astype(str))
        test[name] = (test[comb[0]].astype(str) + '_' +
                      test[comb[1]].astype(str) + '_' +
                      test[comb[2]].astype(str))
        
    elif len(comb) == 4:
        train[name] = (train[comb[0]].astype(str) + '_' +
                       train[comb[1]].astype(str) + '_' +
                       train[comb[2]].astype(str) + '_' +
                       train[comb[3]].astype(str))
        test[name] = (test[comb[0]].astype(str) + '_' +
                      test[comb[1]].astype(str) + '_' +
                      test[comb[2]].astype(str) + '_' +
                      test[comb[3]].astype(str))
    
    encoded_columns.append(name)

train[encoded_columns] = train[encoded_columns].astype('category')
test[encoded_columns] = test[encoded_columns].astype('category')

In [10]:
train.shape

(794868, 113)

In [14]:
# reference: https://www.kaggle.com/code/masayakawamata/imputation-strategies/
train[NUMS] = train[NUMS].fillna(train[NUMS].median())
test[NUMS] = test[NUMS].fillna(train[NUMS].median())

In [15]:
FEATURES = NUMS + CATS + encoded_columns

print(f"Train Shape: {train.shape}")
print(f"Test  Shape: {test.shape}")
train.head(3)

Train Shape: (794868, 113)
Test  Shape: (250000, 112)


,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Num,is_weekend,ELm_r0,ELm_r1,ELm_r2,Episode_Length_minutes_Host_Popularity_percentage,Episode_Length_minutes_Guest_Popularity_percentage,Episode_Length_minutes_Number_of_Ads,Episode_Num_Host_Popularity_percentage,Episode_Num_Guest_Popularity_percentage,Episode_Num_Number_of_Ads,Host_Popularity_percentage_Guest_Popularity_percentage,Host_Popularity_percentage_Number_of_Ads,Host_Popularity_percentage_Episode_Sentiment,Episode_Length_minutes_Podcast_Name,Episode_Num_Podcast_Name,Guest_Popularity_percentage_Podcast_Name,ELm_r1_Episode_Num,ELm_r1_Host_Popularity_percentage,ELm_r1_Guest_Popularity_percentage,ELm_r2_Episode_Num,ELm_r2_Episode_Sentiment,ELm_r2_Publication_Day,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage,Episode_Length_minutes_Episode_Num_Guest_Popularity_percentage,Episode_Length_minutes_Episode_Num_Number_of_Ads,Episode_Length_minutes_Episode_Num_Episode_Sentiment,Episode_Length_minutes_Episode_Num_Publication_Day,Episode_Length_minutes_Host_Popularity_percentage_Guest_Popularity_percentage,Episode_Length_minutes_Host_Popularity_percentage_Number_of_Ads,Episode_Length_minutes_Host_Popularity_percentage_Episode_Sentiment,Episode_Length_minutes_Host_Popularity_percentage_Publication_Day,Episode_Length_minutes_Host_Popularity_percentage_Publication_Time,Episode_Length_minutes_Guest_Popularity_percentage_Number_of_Ads,Episode_Length_minutes_Guest_Popularity_percentage_Publication_Day,Episode_Length_minutes_Guest_Popularity_percentage_Publication_Time,Episode_Length_minutes_Number_of_Ads_Episode_Sentiment,Episode_Length_minutes_Number_of_Ads_Publication_Day,Episode_Length_minutes_Episode_Sentiment_Publication_Time,Episode_Num_Host_Popularity_percentage_Guest_Popularity_percentage,...,Host_Popularity_percentage_Guest_Popularity_percentage_Publication_Day,Host_Popularity_percentage_Guest_Popularity_percentage_Publication_Time,Host_Popularity_percentage_Number_of_Ads_Publication_Day,Guest_Popularity_percentage_Number_of_Ads_Episode_Sentiment,Guest_Popularity_percentage_Number_of_Ads_Genre,ELm_r1_Number_of_Ads_Episode_Sentiment,ELm_r2_Number_of_Ads_Podcast_Name,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Guest_Popularity_percentage,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Number_of_Ads,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Episode_Sentiment,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Publication_Day,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Publication_Time,Episode_Length_minutes_Episode_Num_Host_Popularity_percentage_Genre,Episode_Length_minutes_Episode_Num_Guest_Popularity_percentage_Number_of_Ads,Episode_Length_minutes_Episode_Num_Guest_Popularity_percentage_Episode_Sentiment,Episode_Length_minutes_Episode_Num_Guest_Popularity_percentage_Publication_Day,Episode_Length_minutes_Episode_Num_Guest_Popularity_percentage_Publication_Time,Episode_Length_minutes_Episode_Num_Number_of_Ads_Episode_Sentiment,Episode_Length_minutes_Episode_Num_Number_of_Ads_Publication_Day,Episode_Length_minutes_Episode_Num_Number_of_Ads_Publication_Time,Episode_Length_minutes_Episode_Num_Publication_Day_Publication_Time,Episode_Length_minutes_Episode_Num_Publication_Day_Genre,Episode_Length_minutes_Host_Popularity_percentage_Guest_Popularity_percentage_Number_of_Ads,Episode_Length_minutes_Host_Popularity_percentage_Guest_Popularity_percentage_Episode_Sentiment,Episode_Length_minutes_Host_Popularity_percentage_Guest_Popularity_percentage_Publication_Day,Episode_Length_minutes_Host_Popularity_percentage_Guest_Popularity_percentage_Publication_Time,Episode_Length_minutes_Host_Popularity_percentage_Number_of_Ads_Episode_Sentiment,Episode_Length_minutes_Host_Popularity_percentage_Number_of_Ads_Publication_Day,Episode_Length_minutes_Host_Popularity_p

In [16]:
train.to_csv('../data/interim/train_exp1.csv', index=False)
test.to_csv('../data/interim/test_exp1.csv', index=False)